#*Project 2: Tic Tac Toe Game*#

In [1]:
!pip install fastapi uvicorn pyngrok tensorflow numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import random
import uvicorn
from fastapi import FastAPI
from fastapi.responses import FileResponse
import nest_asyncio
from pyngrok import ngrok
import os

# Initialize FastAPI app
app = FastAPI()

# Tic-Tac-Toe positions mapping
game_board_position = {
    1: (0, 0), 2: (0, 1), 3: (0, 2),
    4: (1, 0), 5: (1, 1), 6: (1, 2),
    7: (2, 0), 8: (2, 1), 9: (2, 2)
}

# Check for winner
def is_winner(board, marker):
    winning_combinations = [
        [(0, 0), (0, 1), (0, 2)], [(1, 0), (1, 1), (1, 2)], [(2, 0), (2, 1), (2, 2)],
        [(0, 0), (1, 0), (2, 0)], [(0, 1), (1, 1), (2, 1)], [(0, 2), (1, 2), (2, 2)],
        [(0, 0), (1, 1), (2, 2)], [(0, 2), (1, 1), (2, 0)]
    ]
    return any(all(board[pos] == marker for pos in combination) for combination in winning_combinations)

# Minimax Algorithm with Alpha-Beta Pruning
def minimax(board, depth, is_maximizing, alpha, beta):
    if is_winner(board, "O"):
        return 10 - depth  # AI Wins
    if is_winner(board, "X"):
        return depth - 10  # Player Wins
    if " " not in board.values():
        return 0  # Draw

    available_moves = [pos for pos in game_board_position.keys() if board[game_board_position[pos]] == " "]

    if is_maximizing:
        best_score = -float("inf")
        for move in available_moves:
            board[game_board_position[move]] = "O"
            score = minimax(board, depth + 1, False, alpha, beta)
            board[game_board_position[move]] = " "
            best_score = max(best_score, score)
            alpha = max(alpha, score)
            if beta <= alpha:
                break
        return best_score
    else:
        best_score = float("inf")
        for move in available_moves:
            board[game_board_position[move]] = "X"
            score = minimax(board, depth + 1, True, alpha, beta)
            board[game_board_position[move]] = " "
            best_score = min(best_score, score)
            beta = min(beta, score)
            if beta <= alpha:
                break
        return best_score

# AI Move Selection using Minimax
def ai_move_nn(board):
    available_moves = [pos for pos in game_board_position.keys() if board[game_board_position[pos]] == " "]
    best_move = None
    best_score = -float("inf")

    for move in available_moves:
        board[game_board_position[move]] = "O"
        move_score = minimax(board, 0, False, -float("inf"), float("inf"))
        board[game_board_position[move]] = " "

        if move_score > best_score:
            best_score = move_score
            best_move = move

    return best_move

# Serve UI
@app.get("/")
def serve_ui():
    return FileResponse("/content/ui.html")

# API Endpoint to process moves
@app.post("/move/")
async def process_move(data: dict):
    board = {game_board_position[i]: data["board"].get(str(i), " ") for i in range(1, 10)}

    # Check if Player (X) won
    if is_winner(board, "X"):
        return {"status": "win", "message": "You Win!"}

    # Check if AI (O) won
    if is_winner(board, "O"):
        return {"status": "lose", "message": "AI Wins!"}

    # Check for Draw
    if " " not in board.values():
        return {"status": "draw", "message": "It's a Draw!"}

    # AI Move - Ensure AI chooses only available moves
    ai_choice = ai_move_nn(board)
    if ai_choice:
        board[game_board_position[ai_choice]] = "O"

    # Check again if AI won after its move
    if is_winner(board, "O"):
        return {"status": "lose", "message": "AI Wins!", "move": ai_choice}

    return {"status": "continue", "move": ai_choice}

# Start FastAPI server with ngrok
ngrok.set_auth_token("2sM1qgfP4EEInKHgHL0KvNoAodo_7Fpnjrv1qcS6X4FfspC2N")  # Replace with your Ngrok Auth Token
public_url = ngrok.connect(8000).public_url
print(f"Public URL: {public_url}")

# Run FastAPI app
nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)

Public URL: https://05c6-34-148-27-17.ngrok-free.app


INFO:     Started server process [429]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     2405:201:301d:a857:6ddd:5537:48e8:3c8f:0 - "GET / HTTP/1.1" 200 OK
INFO:     2405:201:301d:a857:6ddd:5537:48e8:3c8f:0 - "GET /style.css HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [429]
